<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js082_infectionMCS2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [1]:
#@title js082_infectionMCS2D / def exec_html_js() ... exec me first

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js082_infectionMCS2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js082_infectionMCS2D
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2018.01.13 created, last updated on 2018.11.29
//    ver 0.0.1  2019.01.23 v1, last updated on 2021.08.11
//    ver 0.0.2  2021.11.05 v2, last updated on 2021.11.05
//    ver 0.0.3  2023.04.25 v3, last updated on 2023.09.04
//
// -------------------- infection Monte-Carlo simulation MCS2D  -- infection model
//
//  infection model
//
//    particle in field[i][j]
//
//    particle move: random walk : right(E), up(N), left(W), down(S)
//    particle status: non-infected, infected, immune
//      particle status change
//      if the non-infected particle contact an infected particle,
//        status of the particle change to infected.
//      after infected period, particle status change to immune status.
//      immune-particle is not infect.
//      after immune period, particle status change to non-infected status.
//
//    infection model resemble a kind of reaction-diffusion system
//
// --------------------
*/

const infectionMCS2D = (function(){ // ====================  infectionMCS2D Module  ====================

	const g_nMax = 50000;					// array max

	let g_NNp = 30000;						// number of walker
	let g_NNx = 360;						// x-division of the field
	let g_NNy = 360;						// y-division of the field
	let g_sysTime = 0;						// system time
	let g_infectionPeriod = 30;				// infection period
	let g_immunePeriod = 150;				// immune period
	let g_infectionProbability = 0.5;		// infection probability at a contact with another walker
	let g_numberOfNonInfected = 0;			// number of non-infected walker
	let g_numberOfInfected = 0;				// number of infected walker
	let g_numberOfImmune = 0;				// number of immune walker

	const g_field = dimInt2( g_NNx, g_NNy );// field[i][j]  1...NNp:walker number,  0:free space, -1:wall
	const g_status = dimInt1( g_nMax );		// status[i] 0:non-infected 1:infected, 2:immune
	const g_ix = dimInt1( g_nMax );			// ix[i] x-position of i-th walker
	const g_iy = dimInt1( g_nMax );			// iy[i] y-position of i-th walker
	const g_period = dimInt1( g_nMax );		// infected/immune period of i-th walker

	function dimInt1( n ) {
		return new Int32Array( n );
	}

	function dimInt2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( nnp ) {
		g_NNp = nnp;
		g_sysTime = 0;
		clearField();
		setParticles( nnp );
		infect();
	}

	function clearField() {
		const nnx=g_NNx, nny=g_NNy;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				g_field[i][j] = 0;
			}
		}
	}

	function setParticles( nnp ) {
		const nnx=g_NNx, nny=g_NNy;
		for (let ipp=1; ipp<=nnp; ipp++) {
			let x, y;
			do {
				x = Math.floor(nnx*Math.random());
				y = Math.floor(nny*Math.random());
			} while (g_field[x][y]!=0); // field[][] 1.. ipp ..nnp: particle, 0:free space, -1:wall
			g_field[x][y] = ipp;
			g_ix[ipp] = x;
			g_iy[ipp] = y;
			g_status[ipp] = 0; // 0:non-infected 1:infected, 2:immune
			g_period[ipp] = 0;
		}
	}

	function infect() {
		const nnp=g_NNp, nnx0=g_NNx/2, nny0=g_NNy/2;
		for (let ipp=1; ipp<=nnp; ipp++) {
			const x = g_ix[ipp]-nnx0;
			const y = g_iy[ipp]-nny0;
			if ( x*x+y*y<100 ) {
				g_status[ipp] = 1; // infected
				g_period[ipp] = g_infectionPeriod;
			}
		}
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( nCalc, infectionPeriod, immunePeriod ) {
		g_infectionPeriod = infectionPeriod;
		g_immunePeriod = immunePeriod;
		for (let i=0; i<nCalc; i++) {
			timeStep();
		}
		stat();
	}

	function timeStep() {
		const nnp=g_NNp;
		g_sysTime += 1;
		for (let ipp=0; ipp<nnp; ipp++) {
			const st = g_status[ipp]; // status 0:non-infected 1:infected, 2:immune
			if (st==1) { // infected
				g_period[ipp] = g_period[ipp] - 1;
				if (g_period[ipp]<=0) {
					g_status[ipp] = 2;
					g_period[ipp] = g_immunePeriod;
				}
			} else if (st==2) { // immune
				g_period[ipp] = g_period[ipp] - 1;
				if (g_period[ipp]<=0) g_status[ipp] = 0;
			}
			moveWarker(ipp);
		}
	}

	function moveWarker(ipp) {
		const dir = Math.floor(Math.random()*4);
		let ixx, iyy;
		if (dir==0) { // E
			ixx = (g_ix[ipp]+1)%g_NNx; iyy = g_iy[ipp];
		} else if (dir==1) { // W
			ixx = (g_ix[ipp]-1+g_NNx)%g_NNx; iyy = g_iy[ipp];
		} else if (dir==2) { // N
			ixx = g_ix[ipp]; iyy = (g_iy[ipp]+1)%g_NNy;
		} else if (dir==3) { // S
			ixx = g_ix[ipp]; iyy = (g_iy[ipp]-1+g_NNy)%g_NNy;
		}
		const f = g_field[ixx][iyy]; // field[][] 1...NNp: particle, 0:free space, -1:wall
		if (f==0) { // free space
			// move
			g_field[g_ix[ipp]][g_iy[ipp]] = 0;
			g_ix[ipp] = ixx; g_iy[ipp] = iyy;
			g_field[g_ix[ipp]][g_iy[ipp]] = ipp;
		} else if (f>0) { // other particle
			// contact
			if (g_status[ipp]==0 && g_status[f]==1) { // ipp:non-infected and f:infected
				g_status[ipp] = 1;
				g_period[ipp] = g_infectionPeriod;
			}
		} else if (f==-1) { // wall
			// nop
		}
	}

	function stat() {
		const nnp=g_NNp;

		g_numberOfNonInfected = 0;
		g_numberOfInfected = 0;
		g_numberOfImmune = 0;
		for (let ipp=1; ipp<=nnp; ipp++) {
			const st = g_status[ipp];
			if (st==0) g_numberOfNonInfected++;
			if (st==1) g_numberOfInfected++;
			if (st==2) g_numberOfImmune++;
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( nnp )
		evolve:			timeEvolution,			// timeEvolution( nCalc, infectionPeriod, immunePeriod )

		getSysParam:	function() { return [ g_NNp, g_NNx, g_NNy ]; },
		getNow:			function() { return [ g_sysTime, g_numberOfNonInfected, g_numberOfInfected, g_numberOfImmune ]; },
		getWalker:		function(i) { return [ g_status[i], g_ix[i], g_iy[i], g_period[i] ]; },
		getStatus:		function(i) { return g_status[i]; },
		getField:		function(ix,iy) { return g_field[ix][iy]; },
	};

})(); // ====================  infectionMCS2D end  ====================


const js082 = (function(){ // ====================  js Module  ====================

	const theModule = infectionMCS2D;
	const xCanvasSize = 500;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let imageData = null;		// image data
	let canvas;					// canvas2d
	let ctx;					// = canvas.getContext('2d');

	let v_nnp = 30000;			// number of particles
	let v_nCalc = 1;			// 1 (calc/frame)
	let v_infectionPeriod = 30;
	let v_immunePeriod = 150;

	let p_NNp, p_NNx, p_NNy; // <-- theModule.getSysParam()
  let sysTime, numberOfNonInfected, numberOfInfected, numberOfImmune;
  let statusList = [];
  let ixList = [];
  let iyList = [];

	let dispMode = 0; // no use
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
	//let inStepFlag = false;

  let breakFlag = false;
  let perticleFlag = true;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d', { willReadFrequently:true } );
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_nnp );
			imageData = null;
			[ p_NNp, p_NNx, p_NNy ] = theModule.getSysParam();
      perticleFlag = true;
		}

		if ( !pauseFlag ) {
			theModule.evolve( v_nCalc, v_infectionPeriod, v_immunePeriod );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_nCalc, v_infectionPeriod, v_immunePeriod );
			//inStepFlag = true;
		}

		draw( ctx, dispMode );

    if ( perticleFlag ) setParticlesData();

		requestAnimationFrame(animate);
	}

  function setParticlesData() {
    for (let i=0; i<p_NNp; i++) {
			let status, ix, iy, period;
      [ status, ix, iy, period ] = theModule.getWalker(i);
      statusList[i] = status; // status  0:non-infected 1:infected, 2:immune
			ixList[i] = ix;
			iyList[i] = iy;
		}
	}


	// --------------------  draw  --------------------

	const walkerColorData = [ [ 0x00, 0xff, 0x00 ], [ 0xff, 0x00, 0x00 ], [ 0x40, 0x80, 0xff ] ];

	function draw( ctx, dispMode ) {
		const xp = 60, yp = 40, xBoxSize = 360, yBoxSize = Math.floor(xBoxSize/p_NNx*p_NNy);
		[ sysTime, numberOfNonInfected, numberOfInfected, numberOfImmune ] = theModule.getNow();

		// init. imageData
		if ( imageData==null ) {
			imageData = ctx.getImageData(xp, yp, xBoxSize, yBoxSize);
			setImageData();
		}

		// clear
		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		// box
		ctx.strokeStyle = "#888888";
		ctx.strokeRect( xp-1, yp-1, xBoxSize+2, yBoxSize+2 );

		if ( dispMode==0 ) {
			drawField( ctx, xp, yp );
		} else if ( dispMode>0 ) {
			drawZoomedField(ctx, (dispMode+1), xp, yp)
		}
		ctx.fillStyle = walkerColor( 0 );
		ctx.fillText(`normal = ${ numberOfNonInfected }`, 20, 20);
		ctx.fillStyle = walkerColor( 1 );
		ctx.fillText(`infected = ${ numberOfInfected }`, 180, 20);
		ctx.fillStyle = walkerColor( 2 );
		ctx.fillText(`immune = ${ numberOfImmune }`, 360, 20);

		ctx.fillStyle = "#888888";
		ctx.fillText(`field = ${p_NNx} x ${p_NNy},`, 50, yCanvasSize-30);
		ctx.fillText(`number of walkers N = ${p_NNp}`, 240, yCanvasSize-30);
		ctx.fillText(`time = ${sysTime}`, 50, yCanvasSize-10);
	}

	function walkerColor( status ) { // status 0:normal, 1:infected, 2:immune
		const cd = walkerColorData[status];
		return `rgb(${cd[0]},${cd[1]},${cd[2]})`;
	}

	function setImageData() {
		const xSize=p_NNx, ySize=p_NNy, data=imageData.data;

		for (let j=0; j<ySize; j++) {
			for (let i = 0; i<xSize; i++) {
				let idx = (j*xSize+i)*4;
				data[idx] = 0x00;
				data[++idx] = 0x00;
				data[++idx] = 0x00;
				data[++idx] = 0xff;
			}
		}
	}

	function drawField( ctx, xp, yp ) {
		const nnx=p_NNx, nny=p_NNy, data=imageData.data;

		for (let j=0; j<nny; j++) {
			const jj = nny - j - 1;
			for (let i = 0; i<nnx; i++) {
				let idx = (j*nnx+i)*4;
				const f = theModule.getField(i,jj);
				let r = 0, g = 0, b = 0;
				if ( f > 0 ) {
					const st = theModule.getStatus(f); // g_status[f];
					[ r, g, b ] = walkerColorData[st];
				}
				data[idx] = r;
				data[++idx] = g;
				data[++idx] = b;
			}
		}
		ctx.putImageData( imageData, xp, yp );
	}

	function drawZoomedField(ctx, sc, xp, yp) {
		// sc: 2,3,4,5
		const nnx=p_NNx, nny=p_NNy, ic=nnx/2, jc=nny/2, ispan=Math.floor(ic/sc), jspan=Math.floor(jc/sc)
		const i0 = ic-ispan, i1 = ic+ispan, j0 =jc-jspan, j1 = jc+jspan;

		for (let j=j0; j<j1; j++) {
			for (let i = i0; i<i1; i++) {
				const f = theModule.getField(i,j);
				if ( f > 0 ) {
					const color = walkerColor( theModule.getStatus(f) );
					drawDisc( ctx, (i-i0)*sc + xp, (j1-j)*sc + yp, sc*0.5, color );
				}
			}
		}
	}

	function drawDisc( ctx, x, y, r, color ) {
		ctx.fillStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.fill();
	}


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {  // select nnp
		const n = 1 + document.getElementById("slct_theme").selectedIndex;
		v_nnp = n*10000;
		resetFlag = true;
	}

	function setInfectionPeriod() {
		v_infectionPeriod = Number(document.getElementById("range_infp").value);
		document.getElementById("text_infp").innerHTML = " " + v_infectionPeriod;
	}

	function setImmunePeriod() {
		v_immunePeriod = Number(document.getElementById("range_immp").value);
		document.getElementById("text_immp").innerHTML = " " + v_immunePeriod;
	}

	function setDispMode() {  // select dispMode
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function setSpeed() {
		v_nCalc = 1 + document.getElementById("slct_speed").selectedIndex;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, numberOfNonInfected, numberOfInfected, numberOfImmune ];
  }

  function pygetParticlesList() {
    perticleFlag = false;
    return [ statusList, ixList, iyList ]; // status 0:normal, 1:infected, 2:immune
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setInfectionPeriod: setInfectionPeriod, // setInfectionPeriod()
		setImmunePeriod: setImmunePeriod, // setImmunePeriod()
		setDispMode:	setDispMode,	// setDispMode()
		setSpeed:		setSpeed,		// setSpeed()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, numberOfNonInfected, numberOfInfected, numberOfImmune ]
    pygetParticlesList: pygetParticlesList, //() :return [ statusList, ixList, iyList ] // status 0:normal, 1:infected, 2:immune
	};

})(); // ====================  js082 module end  ====================


const js = js082;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js082] infection model - Monte-Carlo simulation MCS2D</p>
<canvas ID="canvas_box" style="background-color: #000000;" WIDTH="500" HEIGHT="480"></canvas>
<br>

<label>number of walker:</label>
<select id="slct_theme" onChange="js.setTheme()">
<option>10000</option><option>20000</option>
<option selected>30000</option><option>40000</option>
</select>
    <span style="margin-right: 130px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>infection period =</label>
<input type="range" id="range_infp" min="0" max="60" value="30" step="1"
style="width:300px" list="tick_infp" oninput="js.setInfectionPeriod()">
<datalist id="tick_infp"><option value="0"><option value="20"><option value="40">
<option value="60"></datalist>
<label id="text_infp"> 30</label>
<br>

<label>immune period =</label>
<input type="range" id="range_immp" min="0" max="300" value="150" step="1"
style="width:300px" list="tick_immp" oninput="js.setImmunePeriod()">
<datalist id="tick_immp"><option value="0"><option value="100"><option value="200">
<option value="300"></datalist>
<label id="text_immp"> 150</label>
<br>

<label>disp mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option>wakers field 360X360</option><option>walkers field zoom x2</option><option>walkers field zoom x3</option>
<option>walkers field zoom x4</option><option>walkers field zoom x5</option>
</select>
    <span style="margin-right: 100px;"></span>
<label>nCalc/frame:</label>
<select id="slct_speed" onChange="js.setSpeed()">
<option selected>1</option><option>2</option><option>3</option><option>4</option>
<option>5</option><option>6</option><option>7</option><option>8</option>
</select>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# exec html-js code, and python control

import time

# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(1)

# get system data and print
for i in range(10):
  [ sysTime, ss, ii, rr ] = eval_js( 'js.pygetData({})'.format(i) )
  print( f'i = {i:>2d},  time = {sysTime:>7.1f},  Normal = {ss:>6d},  Infected = {ii:>6d},  Immune = {rr:>6d} ' )
  time.sleep(3)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# exec html-js code, and python control /  ss, ii, rr plot

import time
import matplotlib.pyplot as plt

timeList = []
ssList = []
iiList = []
rrList = []

# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(1)

# get system data and print
print("-- getting data --")
for i in range(30):
  [ sysTime, ss, ii, rr ] = eval_js( 'js.pygetData({})'.format(i) )
  timeList.append(sysTime)
  ssList.append(ss)
  iiList.append(ii)
  rrList.append(rr)
  time.sleep(1)

# animation loop break to END
eval_js( 'js.breakLoop()' )
print("-- simulator stopped --")


plt.plot( timeList, ssList, 'og-' )
plt.plot( timeList, iiList, 'xr-' )
plt.plot( timeList, rrList, '^b-' )
plt.xlabel('time')
plt.ylabel('number of walkers')
plt.show()

In [ ]:
# exec html-js code, and python control / get walkers data

import time

# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(1)

# get system data and print
for i in range(10):
  [ sysTime, ss, ii, rr ] = eval_js( 'js.pygetData({})'.format(i) )
  print( f'i = {i:>2d},  time = {sysTime:>7.1f},  Normal = {ss:>6d},  Infected = {ii:>6d},  Immune = {rr:>6d} ' )
  time.sleep(1)

# get walkers data
print("")
print("-- get walkers data --")
[ statusList, ixList, iyList ] = eval_js('js.pygetParticlesList()')
print("-- len(statusList) =",len(statusList) , " --")
print("-- len(ixList) =",len(ixList) , " --")
print("-- len(iyList) =",len(iyList) , " --")

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# scatter plot walkers / statusList, ixList, iyList

import numpy as np
import matplotlib.pyplot as plt

colorList = [ (0,1,0), (1,0,0), (0,0,1) ]  # status 0:normal, 1:infected, 2:immune

value = [colorList[s] for s in statusList]

plt.figure(figsize=(8, 8))
plt.scatter(ixList, iyList, s=2, c=value)
